# Create label for Routing

In [ ]:
def rule_based_classify(text):
    # คำสำคัญที่แสดงถึงการคาดการณ์ (prediction)
    prediction_keywords = [
        "อ่านคำถาม","ตอบคำถาม","Read the","Answer the"
    ]

    # ตรวจสอบว่าในข้อความมีคำคาดการณ์หรือไม่
    if any(keyword in text for keyword in prediction_keywords):
        return "multiple"
    return "prediction"

#fucntion to split text by \n and merge after first array with space
#remove specific keyword after join
remove = ["คำถาม","Question",":","Q","บริบท","Context","Answer","คำตอบ"]
def split_text(text):
    text = text.split("\n")
    text = " ".join(text[1:])
    for i in remove:
        text = text.replace(i,"")
    return text

In [ ]:
import pandas as pd

test = pd.read_csv('/home/siamai/data/Focus/agentic/data/test.csv')
# Apply the classifier
test["message_type"] = test["query"].apply(rule_based_classify)
test["message_sliced"] = test["query"].apply(split_text)
test

In [ ]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch

ROUTER_PATH = "/home/siamai/data/chuniji/week8/FinetunedBERT/BERTfine/Onfire"

tokenizer = AutoTokenizer.from_pretrained(ROUTER_PATH)
model_cls = AutoModelForSequenceClassification.from_pretrained(ROUTER_PATH, num_labels=2).cuda()
model_cls.eval()
def classify_question(query: str) -> str:    
    mapping = {0: "multiple_choice", 1: "timeseries"}    
    inputs = tokenizer_cls(query, padding=True, truncation=True, return_tensors="pt").to(model_cls.device)
    outputs = model_cls(**inputs)    
    pred = torch.argmax(outputs.logits, dim=1).item()    
    return mapping.get(pred, "unknown")

In [ ]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer
from datasets import Dataset
import pandas as pd

# Prepare dataset
df = test.copy()
df["labels"] = df["message_type"].apply(lambda x: 1 if x == "prediction" else 0)  # MUST be 'labels'

# Convert to Hugging Face Dataset
dataset = Dataset.from_pandas(df)

# # Load tokenizer and model
# model_name = "xlm-roberta-base"
# tokenizer = AutoTokenizer.from_pretrained(model_name)
# model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=2)

# # Tokenizer function that includes 'labels'
def tokenize(example):
    tokens = tokenizer(example["message_sliced"], truncation=True, padding="max_length")
    tokens["labels"] = example["labels"]  # ✅ Add labels here
    return tokens

# Tokenize
tokenized_dataset = dataset.map(tokenize)

# Split
split_dataset = tokenized_dataset.train_test_split(test_size=0.6)
train_dataset = split_dataset["train"]
eval_dataset = split_dataset["test"]

# Training arguments
training_args = TrainingArguments(
    output_dir=None,
    per_device_train_batch_size=64,
    num_train_epochs=3,
    eval_strategy="steps",
    eval_steps=20,
    save_steps=100,
    save_total_limit=1,
    load_best_model_at_end=True,
    report_to="none"
)

# Optional: Accuracy metric
from sklearn.metrics import accuracy_score,f1_score

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    preds = logits.argmax(axis=1)
    return {"accuracy": accuracy_score(labels, preds),
            "f1_score":f1_score(labels, preds)}

# Trainer
trainer = Trainer(
    model=model_cls,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    compute_metrics=compute_metrics
)

# Train
# trainer.train()

In [ ]:
# model.save_pretrained("../model/xlm_routing")
# tokenizer.save_pretrained("../model/xlm_routing")

In [ ]:
#plot confusion matrix
from sklearn.metrics import confusion_matrix


predictions, _, _ = trainer.predict(eval_dataset)
y_pred = predictions.argmax(axis=1)

cm = confusion_matrix(eval_dataset["labels"], y_pred)
cm

In [ ]:
eval_dataset

In [ ]:
df = pd.DataFrame({"labels":y_pred,
                   "true_labels":eval_dataset["labels"],
                   "query":eval_dataset["query"]
                   })
df

# Call local API

In [4]:
import requests

api_url = "http://localhost:6666/generate" 
input = """Answer the question with the appropriate options A, B, C and D. Please respond with the exact answer A, B, C or D only. Do not be verbose or provide extra information. 
Question: According to COSO, which of the following is the most effective method to transmit a message of ethical behavior throughout an organization?
Answer Choices: A: Demonstrating appropriate behavior by example., B: Strengthening internal audit’s ability to deter and report improper behavior., C: Removing pressures to meet unrealistic targets, particularly for short-term results., D: Specifying the competence levels for every job in an organization and translating those levels to requisite knowledge and skills. 
Answer:"""

data = {
    "prompt": input,
    "temperature": 0.7,
}

response = requests.post(api_url, json=data)

if response.status_code == 200:
    print("Response:", response.json()["generated_text"])
else:
    print(f"Failed with status {response.status_code}: {response.text}")

Response: Answer the question with the appropriate options A, B, C and D. Please respond with the exact answer A, B, C or D only. Do not be verbose or provide extra information. 
Question: According to COSO, which of the following is the most effective method to transmit a message of ethical behavior throughout an organization?
Answer Choices: A: Demonstrating appropriate behavior by example., B: Strengthening internal audit’s ability to deter and report improper behavior., C: Removing pressures to meet unrealistic targets, particularly for short-term results., D: Specifying the competence levels for every job in an organization and translating those levels to requisite knowledge and skills. 
Answer: A: Demonstrating appropriate behavior by example.
Explanation: According to COSO, the most effective method to transmit a message of ethical behavior throughout an organization is by demonstrating appropriate behavior by example. This means that senior leaders and managers should set the t

In [43]:
def classify(query):
  prompt = f"""
  You are a classifier that categorizes a question into one of two types: `multiple` or `prediction`.

  Definitions:

  1. `multiple`:  
  - These questions ask the user to choose the correct answer from a set of options (e.g., A, B, C, D).  
  - They often include phrases like "เลือกตัวเลือกที่เหมาะสม" or "โปรดตอบด้วย A, B, C หรือ D เท่านั้น".

  Example:  
  "ตอบคำถามด้วยตัวเลือกที่เหมาะสม A, B, C และ D โปรดตอบด้วยคำตอบที่ถูกต้อง A, B, C หรือ D เท่านั้น อย่าใช้คำฟุ่มเฟือยหรือให้ข้อมูลเพิ่มเติม  
  คำถาม: ______ สถานที่ทำงานเกี่ยวข้องกับการเสริมสร้างศักยภาพให้พนักงาน ตัวอย่างเช่น 'job enrichment'  
  ตัวเลือกคำตอบ: A: Re-invigorating, B: Re-flourishing, C: Revitalizing, D: Rehumanizing"

  2. `prediction`:  
  - These questions require interpretation of data, forecasting, or estimating outcomes based on patterns.  
  - Often found in financial, statistical, or analytical contexts.

  Example:  
  "วิเคราะห์ข้อมูลและทวีตเพื่อสรุปว่าราคาปิดของ $gs จะปรับตัวขึ้นหรือลงในวันที่ 2017-12-20 โปรดยืนยันว่าขึ้นหรือลง  
  บริบท: วันที่, เปิด, สูง, ต่ำ, ปิด, ปิดปรับ, เพิ่ม 5%, ...  
  2017-12-19, 2.0, 2.1, -0.1, -1.4, -1.4, ..."

  ---

  **Your task:**  
  Classify the following query as either `multiple` or `prediction`.  
  **Only return one of the following two words (in lowercase) without explanation**:  
  - `multiple`  
  - `prediction`

  Query:  
  \"\"\"{query}\"\"\"
  Assistance :
  """


  data = {
  "prompt": prompt,
  "temperature": 0.1}
  response = requests.post(api_url, json=data)
  return response

In [48]:
#randomly select row from dataframe as input
import pandas as pd

df = pd.read_csv("/home/siamai/data/Focus/agentic/data/test.csv")
user_input = df.sample(n=1).iloc[0]["query"]
print(f"User input: {user_input}")
print("-"*50)   
response = classify(user_input)
response.json()["generated_text"].split("\n")[-1]

User input: Answer the question with the appropriate options A, B, C and D. Please respond with the exact answer A, B, C or D only. Do not be verbose or provide extra information. 
Question: Which of the following statements are true concerning maximum likelihood (ML) estimation in the context of GARCH models?

i) Maximum likelihood estimation selects the parameter values that maximise the

probability that we would have actually observed the values of the series y that we

actually did.


ii) GARCH models can only be estimated by ML and not by OLS


iii) For estimation of a standard linear model (with no GARCH), the OLS and ML

estimates for the slope and intercept parameters will be identical but the estimator

for the variance of the disturbances is slightly different


iv) Most computer packages use numerical procedures to estimate GARCH models

rather than a set of analytical formulae
Answer Choices: A: (ii) and (iv) only, B: (i) and (iii) only, C: (i), (ii), and (iii) only, D: (i

'Assistant: The US Global Jets ETF (JETS) is a fund that invests in companies that are active in the global commercial aerospace industry. Its holdings include companies that manufacture and service commercial aircraft, aircraft parts and equipment, as well as companies that provide ground support services and other'

# Vllm

In [44]:
def classify(query):
  prompt = f"""
  You are a classifier that categorizes a question into one of two types: `multiple` or `prediction`.

  Definitions:

  1. `multiple`:  
  - These questions ask the user to choose the correct answer from a set of options (e.g., A, B, C, D).  
  - They often include phrases like "เลือกตัวเลือกที่เหมาะสม" or "โปรดตอบด้วย A, B, C หรือ D เท่านั้น".

  Example:  
  "ตอบคำถามด้วยตัวเลือกที่เหมาะสม A, B, C และ D โปรดตอบด้วยคำตอบที่ถูกต้อง A, B, C หรือ D เท่านั้น อย่าใช้คำฟุ่มเฟือยหรือให้ข้อมูลเพิ่มเติม  
  คำถาม: ______ สถานที่ทำงานเกี่ยวข้องกับการเสริมสร้างศักยภาพให้พนักงาน ตัวอย่างเช่น 'job enrichment'  
  ตัวเลือกคำตอบ: A: Re-invigorating, B: Re-flourishing, C: Revitalizing, D: Rehumanizing"

  2. `prediction`:  
  - These questions require interpretation of data, forecasting, or estimating outcomes based on patterns.  
  - Often found in financial, statistical, or analytical contexts.

  Example:  
  "วิเคราะห์ข้อมูลและทวีตเพื่อสรุปว่าราคาปิดของ $gs จะปรับตัวขึ้นหรือลงในวันที่ 2017-12-20 โปรดยืนยันว่าขึ้นหรือลง  
  บริบท: วันที่, เปิด, สูง, ต่ำ, ปิด, ปิดปรับ, เพิ่ม 5%, ...  
  2017-12-19, 2.0, 2.1, -0.1, -1.4, -1.4, ..."

  ---

  **Your task:**  
  Classify the following query as either `multiple` or `prediction`.  
  **Only return one of the following two words (in lowercase) without explanation**:  
  - `multiple`  
  - `prediction`

  Query:  
  \"\"\"{query}\"\"\"
  Assistance :
  """
  response = client.chat.completions.create(
    model="KBTG-Labs/THaLLE-0.1-7B-fa",
    messages=[{"role": "user", "content": f"{query}"}]
)
  return response.choices[0].message.content

In [45]:
from openai import OpenAI
client = OpenAI(base_url="http://0.0.0.0:3000/v1", api_key="focus-deploy")
response = client.chat.completions.create(
    model="KBTG-Labs/THaLLE-0.1-7B-fa",
    messages=[{"role": "user", "content": f"Hello"}]
)
print(response.choices[0].message.content)

Hello! How can I assist you today?


In [50]:
user_input = """ตอบคำถามด้วยตัวเลือกที่เหมาะสม A, B, C และ D โปรดตอบด้วยคำตอบที่ถูกต้อง A, B, C หรือ D เท่านั้น อย่าใช้คำฟุ่มเฟือยหรือให้ข้อมูลเพิ่มเติม

คำถาม: ______ สถานที่ทำงานเกี่ยวข้องกับการเสริมสร้างศักยภาพให้พนักงาน ตัวอย่างเช่น 'job enrichment' ที่พนักงานได้รับขอบเขตที่ใหญ่ขึ้นในการตัดสินใจว่าจะจัดระเบียบงานของตนอย่างไร หรือ 'job enlargement' ที่พนักงานได้รับมอบหมายงานที่หลากหลายมากขึ้น

ตัวเลือกคำตอบ: A: Re-invigorating, B: Re-flourishing, C: Revitalizing, D: Rehumanizing

คำตอบ:"""
classify(user_input)

'C: Revitalizing'